In [51]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import ensemble  #for random forest
from sklearn.cross_validation import train_test_split

In [830]:
#import the three data tables - employees.csv, headers.csv, public_schools.csv
#and merge them by county,district and school codes
employees_df = pd.read_csv("employees.csv")
headers_df = pd.read_csv("headers.csv")
schools_df = pd.read_csv("public_schools.csv")
print employees_df.shape, headers_df.shape, schools_df.shape

(144158, 20) (2461, 7) (2662, 49)


In [831]:
#clean up some of the values so that they can be joined with the school headers data:
#employee text values do not have commas or apostrophes so remove them from the school data values
headers_df.school = headers_df.school.str.replace("'",'')
headers_df.district = headers_df.district.str.replace("'",'')
schools_df.school_name = schools_df.school_name.str.replace("'",'')
schools_df.district_name = schools_df.district_name.str.replace("'",'')

#correct the fact that 7s appeared as CE in the employee file
employees_df.school = employees_df.school.str.replace('RAHWAY CETH','RAHWAY 7TH')
employees_df.school = employees_df.school.str.replace('ROOSEVELT SCHOOL #CE','ROOSEVELT SCHOOL #7')
employees_df.school = employees_df.school.str.replace('ANTONIA PANTOJA SCHOOL NO. 2CE','ANTONIA PANTOJA SCHOOL NO. 27')
employees_df.school = employees_df.school.str.replace('#1CE','#17')
employees_df.school = employees_df.school.str.replace('SCHOOL 2CE','SCHOOL 27')
employees_df.school = employees_df.school.str.replace('ULYSSES S. GRANT SCHOOL # CE','ULYSSES S. GRANT SCHOOL # 7')
employees_df.school = employees_df.school.str.replace('SCHOOL CE','SCHOOL 7')
employees_df.school = employees_df.school.str.replace('TERENCE C. REILLY SCHOOL # CE','TERENCE C. REILLY SCHOOL # 7')
employees_df.school = employees_df.school.str.replace('SCHOOL # 1CE','SCHOOL # 17')
employees_df.school = employees_df.school.str.replace('BELLEVILLE PSCE','BELLEVILLE PS7')

#correct various mismatches in the data:
employees_df.district = employees_df.district.str.replace(' VC',' VOCATIONAL')

headers_df.school.loc[(headers_df.school == 'LINCOLN ELEMENTARY SCHOOL') & (headers_df.district == 'KEARNY TOWN')] \
                      = 'LINCOLN MIDDLE SCHOOL'
headers_df.school.loc[(headers_df.school == 'SCHOOL OF SCIENCE TECHNOLOGY ENGG & MATHEMATICS') & \
                     (headers_df['district'] == 'PATERSON CITY')] \
                    = 'SCHOOL OF SCIENCE TECHNOLOGY ENGINEERING AND MATHEMATICS'

schools_df.district_code.loc[(schools_df['school_name'] == 'LAMBERTVILLE PUBLIC SCHOOL')] = 2530
schools_df.school_code.loc[(schools_df['school_name'] == 'LAMBERTVILLE PUBLIC SCHOOL')]= 50
schools_df.school_code.loc[(schools_df['school_name'] == 'HENRY L. BONSALL FAMILY SCHOOL')] = 100
schools_df.school_code.loc[(schools_df['school_name'] == 'FRANCIS X. MCGRAW ELEMENTARY')] = 230
schools_df.school_code.loc[(schools_df['school_name'] == 'RAFAEL CORDERO MOLINA ELEMENTARY SCHOOL')] = 250
schools_df.school_code.loc[(schools_df['school_name'] == 'JOHN GREENLEAF WHITTIER FAMILY SCHOOL')] = 340
schools_df.district_code.loc[(schools_df['school_name'] == 'SOUTH HUNTERDON HIGH SCHOOL')] = 4890
schools_df.district_code.loc[(schools_df['school_name'] == 'STOCKTON BOROUGH SCHOOL')] = 5050
schools_df.school_code.loc[(schools_df['school_name'] == 'STOCKTON BOROUGH SCHOOL')]  = 50

    
headers_df = pd.merge(headers_df, schools_df, how='left', on=['county_code','district_code', 'school_code'])

remaining = pd.DataFrame([
    ['GRETTA R. OSTROVSKY MIDDLE SCHOOL',    -74.08466,     40.84146],
    ['JR/SR HIGH SCHOOL',    -74.81260,     40.04495],
    ['EAST CAMDEN MIDDLE SCHOOL',    -75.12285,     39.95576],
    ['PYNE POYNT MIDDLE SCHOOL',    -75.12696,    39.93595],
    ['STOCKTON BOROUGH SCHOOL DISTRICT',    -74.49987,     40.16706],
    ['WEST AMWELL TOWNSHIP',   -74.11532,     40.91506],
    ['MONUMENT ELEMENTARY SCHOOL',   -74.71268,     40.26977],
    ['AUDREY W. CLARK ELEMENTARY SCHOOL',    -73.98236,     40.30576],
    ['BOUND BROOK ELEMENTARY SCHOOL',    -74.47620,     40.58596],
    ['VISIONS ACADEMY CHARTER HIGH SCHOOL',    -74.49987,     40.16706],
    ['CENTRAL JERSEY ARTS CHARTER SCHOOL',   -75.50865,     39.66265],
    ['GALLOWAY COMMUNITY CHARTER SCHOOL',    -74.49987,     40.16706]])
remaining.columns = ['school','latitude','longitude']
#print remaining
for i in range(remaining.shape[0]):
    headers_df.latitude.loc[headers_df['school'] == remaining.school.iloc[i]] = remaining.latitude.iloc[i]
    headers_df.longitude.loc[headers_df['school'] == remaining.school.iloc[i]] = remaining.longitude.iloc[i]
    
print headers_df.shape
probs = pd.isnull(headers_df.latitude).nonzero()[0]
print len(probs)
print probs
headers_df.iloc[probs]

headers_df.school.loc[headers_df.school == 'WEST MILFORD TOWNSHIP HIGH SCHOOL'] = 'WEST MILFORD HIGH SCHOOL'
headers_df.school.loc[headers_df.school == 'HIGH POINT REGIONAL HS'] ='HIGH POINT REGIONAL HIGH SCHOOL'
headers_df.school.loc[headers_df.school == 'SUMMIT SENIOR HIGH SCHOOL'] = 'SUMMIT HIGH SCHOOL'
headers_df.school.loc[headers_df.school == 'HAMBURG BORO'] ='HAMBURG SCHOOL'
headers_df.school.loc[headers_df.school == 'MAHWAH HIGH'] ='MAHWAH HIGH SCHOOL'
headers_df.school.loc[headers_df.school == 'JOYCE KILMER'] = 'JOYCE KILMER SCHOOL'
headers_df.school.loc[headers_df.school == 'BETSY ROSS'] = 'BETSY ROSS ELEMENTARY'
headers_df.school.loc[headers_df.school == 'GEORGE WASHINGTON'] = 'GEORGE WASHINGTON SCHOOL'
employees_df.school.loc[employees_df.school == 'BULINGTON CITY HIGH SCHOOL'] = 'BURLINGTON CITY HIGH SCHOOL'
headers_df.school.loc[headers_df.school == 'CLEMENTON ELEMENTARY SCHOOL'] = 'CLEMENTON ELEMENTARY SCHOOL DISTRICT'
headers_df.school.loc[headers_df.school == 'ALEXANDER DENBO SCHOOL'] = 'ALEXANDER DENBO/ALETTA CRICHTON SCHOOL'
headers_df.school.loc[headers_df.school == 'SOMERDALE PARK SCHOOL'] = 'SOMERDALE SCHOOL DISTRICT'
headers_df.school.loc[headers_df.school == 'MAURICE RIVER TOWNSHIP ELEMENTARY SCHOOL'] = 'MAURICE RIVER TOWNSHIP SCHOOL DISTRICT'
headers_df.school.loc[headers_df.school == 'FREEDOM ACADEMY CHARTER SCHOOL'] = 'FREEDOM PREP CHARTER SCHOOL'
headers_df.school.loc[headers_df.school == 'LINCOLN/ROOSEVELT SCHOOL'] = 'LINCOLN/ROOSEVELT SCHOOL DISTRICT'
headers_df.school.loc[headers_df.school == 'NIXON ELEMENTARY SCHOOL'] = 'NIXON ELEMENTARY SCHOOL DISTRICT'
headers_df.school.loc[headers_df.school == 'ROXBURY HIGH SCHOOL'] = 'ROXBURY HIGH SCHOOL DISTRICT'
headers_df.school.loc[headers_df.school == 'KITTATINNY REGIONAL SCHOOL'] = 'KITTATINNY HIGH SCHOOL DISTRICT'

#merge the salary data with the combined school datafiles
salaries = pd.merge(employees_df, headers_df, how='left', on=['county','district', 'school'])



(2568, 53)
0
[]


In [832]:
#check how many problem records (i.e. null latitudes) remain
print salaries.shape
probs = pd.isnull(salaries.latitude).nonzero()[0]
print len(probs)
salaries.iloc[probs]
probs


(149196, 70)
9026


array([    40,     41,     42, ..., 149097, 149103, 149126])

In [833]:
#what to do with those DISTRICT OFFICE values in school?  
#will impute the mean lat and long for the district and county.
dist_office = salaries['school'].map(lambda x: x =='DISTRICT OFFICE')
print sum(dist_office)

6474


In [834]:
avg_lat_per_county_dist = salaries[-dist_office].groupby(['county','district']).mean()

salaries['county_district'] = zip(salaries.county, salaries.district)

#get the indices of the district office records
dist_office_idxs = dist_office.nonzero()[0]
print len(dist_office_idxs)
print dist_office_idxs[range(20)]
print "before set: ", salaries.latitude[41], salaries.longitude[41]
#for each district office record, get the avg latitude for that county, district combination and put into a series
#then do the same for longitude
avg_lat_for_dist_office_recs = salaries.county_district.iloc[dist_office_idxs].map(lambda x: \
                                                               avg_lat_per_county_dist.latitude.loc[x])
avg_long_for_dist_office_recs = salaries.county_district.iloc[dist_office_idxs].map(lambda x: \
                                                               avg_lat_per_county_dist.longitude.loc[x])

salaries.latitude.iloc[dist_office_idxs] = avg_lat_for_dist_office_recs
salaries.longitude.iloc[dist_office_idxs] = avg_long_for_dist_office_recs

print "after set: ", salaries.latitude[41], salaries.longitude[41]

6474
[ 40  41  42  44  45  46  47  48  49  92  93  94  95 107 108 119 120 121
 128 129]
before set:  nan nan
after set:  40.3288218554 -74.7970826145


In [835]:
#check how many problem records (i.e. null latitudes) remain
print salaries.shape
probs_after = pd.isnull(salaries.latitude).nonzero()[0]
print len(probs_after)
#probs_after


(149196, 71)
2564


In [829]:
#this doesn't work so ignoring for now.  The point was to
#see if any of the school names in employee can be found in public_schools.csv if they were not found in header.csv
schools_df['district_school'] = zip(schools_df.district_name,schools_df.school_name)
#print schools_df.loc[(schools_df.district_name == 'PASSAIC CITY SCHOOL DISTRICT') & (schools_df.school_name == 'MARIO DRAGO SCHOOL # 3')]
print "here:"
#print schools_df.loc[(schools_df.district_name == 'PASSAIC CO ED SERV COMM') & (schools_df.school_name == 'HOPE ACADEMY')]

salaries['district_school'] = zip(salaries.district, salaries.school)

#get the indices of the null latitude records
null_lat_idxs = pd.isnull(salaries.latitude).nonzero()[0]
print len(null_lat_idxs), null_lat_idxs
print "ex: ", salaries.district_school.iloc[169]
print "before setd: ", salaries.latitude.iloc[169], salaries.longitude.iloc[169]
print "before setd 2225: ", salaries.latitude.iloc[2225], salaries.longitude.iloc[2225]
#for each null latitude record, get the latitude for that district and school combination in schools_df
#and put into a series
#then do the same for longitude
new_lats_for_null_lat_recs = salaries.district_school.iloc[null_lat_idxs].map(lambda x: \
                                                    schools_df.latitude.loc[schools_df.school_name == x[1]])
print len(new_lats_for_null_lat_recs)
new_longs_for_null_long_recs = salaries.district_school.iloc[null_lat_idxs].map(lambda x: \
                                                    schools_df.longitude.loc[schools_df.school_name == x[1]])

print len(new_longs_for_null_long_recs)
salaries.latitude.iloc[null_lat_idxs] = new_lats_for_null_lat_recs
salaries.longitude.iloc[null_lat_idxs] = new_longs_for_null_long_recs

print "after setd: ", salaries.latitude.iloc[169], salaries.longitude.iloc[169]
print "after setd 2225: ", salaries.latitude.iloc[2225], salaries.longitude.iloc[2225]
null_lat_idxs = pd.isnull(salaries.latitude).nonzero()[0]
print len(null_lat_idxs), null_lat_idxs

here:
2564 [   169   2225   3780 ..., 149097 149103 149126]
ex:  ('PASSAIC CO ED SERV COMM', 'HOPE ACADEMY')
before setd:  nan nan
before setd 2225:  nan nan
2564
2564
after setd:  1122    40.869868
Name: latitude, dtype: float64 1122   -74.125379
Name: longitude, dtype: float64
after setd 2225:  Series([], Name: latitude, dtype: float64) Series([], Name: longitude, dtype: float64)
0 []


In [205]:
#salaries.max()

In [206]:
#salaries.min()

In [711]:
#print salaries.job_name.unique()

In [836]:
#clean up the X values in classes_taught
#so that I can convert the column to int:
salaries.classes_taught.loc[salaries.classes_taught > 'A'] = '99'


In [208]:
print salaries.Classes_Taught.unique()
print salaries.Classes_Taught.unique().size

['5' nan '1' '6' '7' '3' '2' '4' '99' '13' '12' '15' '24' '21' '14' '8'
 '18' '26' '11' '20' '22' '10' '9' '35' '31' '25' '29' '30' '17' '19' '27'
 '28' '38' '23' '16' '32' '40' '39' '36' '37' '33' '34']
42


In [837]:
#not sure what to do with nan classes_taught, can set them to 0 if we want:
nan_idxs = pd.isnull(salaries.classes_taught).nonzero()[0]
nan_idxs
salaries.classes_taught.iloc[nan_idxs] = 0
salaries.classes_taught = salaries.classes_taught.astype(int)

In [695]:
print salaries.classes_taught.unique()
print salaries.classes_taught.unique().size

[ 0  1 20 31 15 12  7  5  6  2  3 11  4  8 21 10 17  9 23 14 24 16 32 25 13
 18 19 27 34 28 26 22 36 29 40 30 33 35 99 39 38 37]
42


In [838]:
#will impute the X(now 99) values with mean classes_taught for the job name.
criterion = salaries.classes_taught.map(lambda x: x == 99) 
print sum(criterion)
avg_num_classes_per_job = salaries[-criterion].groupby(['job_name']).mean()
#print type(avg_num_classes_per_job)
#print avg_num_classes_per_job.head(5)
#get the indices of the X records
X_idxs = criterion.nonzero()[0]
print X_idxs[range(20)]
print "before set: ", salaries.classes_taught[24028], salaries.classes_taught[1]
#for each X record, get the avg classes for that job name and put into a series
avg_num_classes_for_X_recs = salaries.job_name.iloc[X_idxs].map(lambda x: \
                                                                round(avg_num_classes_per_job.classes_taught.loc[x]))
#print avg_num_classes_for_X_recs.shape
salaries.classes_taught.iloc[X_idxs] = avg_num_classes_for_X_recs
print "after set: ", salaries.classes_taught[24028], salaries.classes_taught[1]

425
[23610 23687 23689 23690 23701 23705 23706 23708 23945 23946 23947 23948
 23949 23950 23951 24027 24028 24029 24030 24074]
before set:  99 0
after set:  2 0


In [841]:
print salaries.classes_taught.unique()
print salaries.classes_taught.unique().size

[ 0  1 20 31 15 12  7  5  6  2  3 11  4  8 21 10 17  9 23 14 24 16 32 25 13
 18 19 27 34 28 26 22 36 29 40 30 33 35 39 38 37]
41


In [701]:
#print salaries.fte.unique()
#print salaries.fte.unique().size

In [49]:
#salaries.duplicated().sum()

0

In [842]:
#divide salary data into train and test:
sals_train, sals_test = train_test_split(salaries, test_size=0.20, random_state=21)
print sals_train.shape
print sals_test.shape

(119356, 71)
(29840, 71)
